# Анализ тональности (word2vec)

Будем проводить анализ тональности текста. Вся работа будет проводиться на данных <a href="https://ai.stanford.edu/~amaas/data/sentiment/">Large Movie Review Dataset v1.0</a>. Описание данных на английском языке:

*This dataset contains movie reviews along with their associated binary sentiment polarity labels. It is intended to serve as a benchmark for sentiment classification.*

## word2vec

Описание модели `word2vec` можно найти <a href="https://habr.com/ru/post/446530/">здесь</a>. Я же приведу её реализацию на `TensorFlow`. Сама модель `word2vec` нам нужна из тех соображений, что мы не хотим работать с данными большой размерности. Благодаря ей можно получить эмбеддинги (i. e. векторные представления) слов гораздо меньше размерности, чем представления `CountVectorizer` и `TF-iDF`. 

Импортируем инструменты

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

Импортируем данные

In [2]:
data = pd.read_parquet("large_movie_review_dataset.parquet")
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    50000 non-null  object
 1   label   50000 non-null  object
dtypes: object(2)
memory usage: 1.1+ MB


,text,label
0,Bromwell High is a cartoon comedy. It ran at t...,positive
1,Homelessness (or Houselessness as George Carli...,positive
2,Brilliant over-acting by Lesley Ann Warren. Be...,positive
3,This is easily the most underrated film inn th...,positive
4,This is not the typical Mel Brooks film. It wa...,positive


Составляем корпус слов и вектор ответов

In [3]:
corpus = data["text"].apply(lambda t: t.lower()).values
y = data["label"].map({"negative": 0, "positive": 1}).values

Разбиваем выборку на обучающую и тестовую

In [10]:
corpus_train, corpus_test, labels_train, labels_test = train_test_split(corpus, y, test_size=0.25, random_state=42)

**Теперь необходимо подсчитать $n$-граммы**. Для этого будем использовать `CountVectorizer`. Поскольку мои вычислительные мощности пока что ограничены локальным компьютером, ограничимся биграммами в датасете. В дальнейшем это изменится. 

In [5]:
%%time

unigrams = CountVectorizer(ngram_range=(1, 1)).fit(corpus_train)
bigrams = CountVectorizer(ngram_range=(2, 2)).fit(corpus_train)

Wall time: 25.4 s


Теперь составляем датасет для обучения нашей нейросети. Датасет будет составляться по биграммам, на входе первое слово из биграммы, на выходе второе. Примеры в датасете все положительные

In [8]:
%%time

X_train_str = []
y_train_str = []

for bigram in bigrams.vocabulary_:
    words = bigram.split(" ")
    X_train_str += words
    y_train_str += words[::-1]

Wall time: 1.21 s


In [9]:
X_train_str[:4], y_train_str[:4]

(['really', 'don', 'don', 'want'], ['don', 'really', 'want', 'don'])

In [11]:
X_train = unigrams.transform(X_train_str)
X_train.sort_indices()

y_train = unigrams.transform(y_train_str)
y_train.sort_indices()

X_train.shape, y_train.shape

((3861446, 90321), (3861446, 90321))

Теперь обучаем модель. Импортируем инструменты

In [18]:
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Input

Задаём параметры

In [13]:
embedding_size = 300
vocabulary_size = len(unigrams.vocabulary_)
print(vocabulary_size)

90321


Описываем модель нейросети

In [24]:
word_to_vec = Sequential(name="word_to_vec")
word_to_vec.add(Input(shape=(vocabulary_size,)))
word_to_vec.add(Dense(embedding_size, name="embedding", dtype=np.float16)) 
word_to_vec.add(Dense(vocabulary_size, activation="softmax", name="context", dtype=np.float16))
word_to_vec.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [25]:
word_to_vec.fit(X_train, y_train, epochs=5)

Epoch 1/5


TypeError: in user code:

    D:\anaconda3\envs\modern-problems-of-applied-math-and-computer-science\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    D:\anaconda3\envs\modern-problems-of-applied-math-and-computer-science\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    D:\anaconda3\envs\modern-problems-of-applied-math-and-computer-science\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    D:\anaconda3\envs\modern-problems-of-applied-math-and-computer-science\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    D:\anaconda3\envs\modern-problems-of-applied-math-and-computer-science\lib\sit